# CATCHM [DEMO]

CATCHM combines Deepwalk [Perozzi et al., 2014], a network representation learning algorithm, with a powerful boosted tree model for fraud detection (XGBoost). The transductive deepwalk model is augmented with an inductive pooling extension, which enables online fraud detection without lengthy retraining.

This demo notebook contains an overview of the CATCHM approach. The following code sources are used:
- Deepwalk: https://pypi.org/project/nodevectors/
- XGBoost: https://pypi.org/project/xgboost/
- Inductive extension: https://pypi.org/project/fucc/

Before running the demo, please download the demo dataset from Kaggle:
https://www.kaggle.com/ranjeetshrivastav/fraud-detection-dataset

For a baseline without representation learning, please refer to the Pagerank demo notebook

-----

In [1]:
from catchm.embeddings import InductiveDeepwalk

In [2]:
import os
import numpy as np
import pandas as pd
import networkx as nx
from nodevectors import Node2Vec
import xgboost as xgb
from fucc.inductive_step import inductive_pooling
from fucc.metrics import plot_ap, get_optimal_f1_cutoff, get_confusion_matrix
from sklearn.metrics import average_precision_score
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Parameters
dimensions = 32
walk_len = 80
walk_num = 10
window_size = 5
# the 'workers' parameter is used for multi-processing.
workers = 8

## Load Data

In [4]:
### PATH TO DEMO DATA ###
demo_data_path = './transactions/transactions.txt'

In [5]:
df = pd.read_json(demo_data_path,  lines=True, convert_dates=[4])

In [6]:
# Transform fourth column to datetime format
df.iloc[:, 4] = pd.to_datetime(df.iloc[:, 4])

In [7]:
# Sort dataframe by datetime
df = df.sort_values('transactionDateTime')
# Create a transaction ID
df.loc[:, 'TX_ID'] = range(df.shape[0])

In [8]:
# Rename columns to work with hard-coded feature names in our code
df = df.rename(columns={"merchantName":"TERM_MIDUID", "customerId":"CARD_PAN_ID", "isFraud": "TX_FRAUD" })

- **TERM_MIDUID**: beneficiary of the transaction
- **CARD_PAN_ID**: customer initiating the transaction
- **TX_FRAUD**: fraud label

In [9]:
# Split into train and test set
df_train = df.iloc[:400000]
df_test = df.iloc[400000:500000]

## Create network

In [10]:
edgelist = []
for i, row in df_train.iterrows():
    edgelist.append((str(row.CARD_PAN_ID), str(row.TERM_MIDUID)))

In [11]:
from catchm.embeddings import InductiveDeepwalk


In [12]:
IndDeep = InductiveDeepwalk(dimensions=32, walk_len = walk_len, walk_num=walk_num, head_node_type='transfer', workers=4)

In [13]:
from sklearn.pipeline import Pipeline
import xgboost as xgb

In [14]:
y_train = df_train.TX_FRAUD
model = xgb.XGBClassifier(eval_metric = ['auc','aucpr', 'logloss'], n_estimators=300, n_jobs=8, learning_rate=0.1, seed=42, colsample_bytree = 0.6, colsample_bylevel=0.9, subsample = 0.9)

In [15]:
pipe = Pipeline([('embedder', IndDeep), ('model', model)])

In [16]:
pipe.fit(edgelist, y_train)

Making walks... Done, T=88.57
Mapping Walk Names... 

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Done, T=338.38
Training W2V... 

INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 800000 words, keeping 256920 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 1600000 words, keeping 351045 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 2400000 words, keeping 386439 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 3200000 words, keeping 399535 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 4000000 words, keeping 404410 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 4800000 words, keeping 406213 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 5600000 words, keeping 406933 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #80000, processed 6400000 words, keeping 407181 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #90000, processed 7200000 words, keeping 407273 word types
INFO:gensim.models.w

INFO:gensim.models.word2vec:PROGRESS: at sentence #760000, processed 60800000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #770000, processed 61600000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #780000, processed 62400000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #790000, processed 63200000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #800000, processed 64000000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #810000, processed 64800000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #820000, processed 65600000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #830000, processed 66400000 words, keeping 407337 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #840000, processed 67200000 words, keeping 407337 word types
I

...

INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 25.97% examples, 1228677 words/s, in_qsize 18, out_qsize 5
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 26.38% examples, 1229268 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 26.80% examples, 1230618 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 27.19% examples, 1230078 words/s, in_qsize 24, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 27.58% examples, 1229896 words/s, in_qsize 24, out_qsize 2
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 27.98% examples, 1229577 words/s, in_qsize 18, out_qsize 5
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 28.37% examples, 1229510 words/s, in_qsize 21, out_qsize 2
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 28.77% examples, 1229481 words/s, in_qsize 21, out_qsize 2
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 29.16% examples, 1229397 words/s,

INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 54.39% examples, 1222295 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 54.78% examples, 1222190 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 55.18% examples, 1222018 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 55.58% examples, 1222047 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 55.97% examples, 1222007 words/s, in_qsize 22, out_qsize 2
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 56.36% examples, 1221805 words/s, in_qsize 23, out_qsize 3
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 56.77% examples, 1221962 words/s, in_qsize 24, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 57.16% examples, 1221790 words/s, in_qsize 18, out_qsize 5
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 57.55% examples, 1221722 words/s,

INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 83.04% examples, 1222603 words/s, in_qsize 21, out_qsize 2
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 83.44% examples, 1222483 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 83.84% examples, 1222417 words/s, in_qsize 23, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 84.25% examples, 1222402 words/s, in_qsize 24, out_qsize 5
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 84.64% examples, 1222415 words/s, in_qsize 24, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 85.04% examples, 1222479 words/s, in_qsize 24, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 85.43% examples, 1222424 words/s, in_qsize 24, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 85.82% examples, 1222256 words/s, in_qsize 19, out_qsize 4
INFO:gensim.models.base_any2vec:EPOCH 5 - PROGRESS: at 86.21% examples, 1222220 words/s,

Done, T=1436.29


Pipeline(memory=None,
         steps=[('embedder',
                 InductiveDeepwalk(dimensions=32, epochs=5,
                                   head_node_type='transfer', walk_len=80,
                                   walk_num=10, window_size=5, workers=4)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=0.9, colsample_bynode=1,
                               colsample_bytree=0.6,
                               eval_metric=['auc', 'aucpr', 'logloss'], gamma=0,
                               gpu_id=-1, importance_type='gain',
                               interaction_constraints='', learning_rate=0.1,
                               max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=300,
                               n_jobs=8, num_parallel_tree=1,
                               

In [17]:
# TEST data
edgelist_test = []
for i, row in df_test.iterrows():
    edgelist_test.append((str(row.CARD_PAN_ID), str(row.TERM_MIDUID)))
    
y_test = df_test.TX_FRAUD

In [18]:
y_pred_proba = pipe.predict_proba(edgelist_test)[:, 1]

100%|██████████| 25000/25000 [00:24<00:00, 1039.27it/s]


## Deepwalk

Fitting the Deepwalk model to the network can take a while depending on your local workstation and the number of 'workers' used for multiprocessing. 

In [ ]:
# Fit embedding model to graph
# Node2Vec with p,q=1 is identical to Deepwalk
g2v = Node2Vec(
    n_components=dimensions,
    walklen = walk_len,
    epochs = walk_num,
    w2vparams={'workers': workers, 'window': window_size}
)

g2v.fit(G)
model = g2v.model

In [ ]:
# Retrieve for each transaction the associated embedding
embeddings = {}
for i in df_train.TX_ID:
    embeddings[i] = model.wv[str(i)]


embeddings = pd.DataFrame().from_dict(embeddings, orient='index')

In [ ]:
# Merge training data with the generated embeddings
df_train = df_train.merge(embeddings, left_on='TX_ID', right_index=True)

In [ ]:
df_train.head()

## Inductive Pooling

In [ ]:
# Apply inductive mean pooling
results = inductive_pooling(df=df_test, embeddings=embeddings, G=G, workers=workers)

In [ ]:
df_new_embeddings = pd.concat([pd.DataFrame(li).transpose() for li in results])

In [ ]:
# Merge test data with the inductively generated embeddings
df_new_embeddings.index = df_test.TX_ID
df_test = df_test.merge(df_new_embeddings, left_on='TX_ID', right_index=True)

## XGBoost Classifier

In [ ]:
# Only use the embeddings as input features for XGBoost
embedding_features = [i for i in range(dimensions)]

In [ ]:
# Final 20% of training data is used as validation set
X_train = df_train[embedding_features].iloc[:int(df_train.shape[0]*0.8)]
X_val = df_train[embedding_features].iloc[int(df_train.shape[0]*0.8):]
y_train = df_train.TX_FRAUD.iloc[:int(df_train.shape[0]*0.8)]
y_val = df_train.TX_FRAUD.iloc[int(df_train.shape[0]*0.8):]

X_test = df_test[embedding_features]
y_test = df_test.TX_FRAUD

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# These parameters are not necessarily optimal! Hyperparameter tuning could further improve performance.
xgb_params = {
    'eval_metric': ['auc','aucpr', 'logloss'],
    'objective':'binary:logistic',
    'n_estimators': 300,
    'n_jobs':8,
    'learning_rate':0.1,
    'seed':42,
    'colsample_bytree':0.6,
    'colsample_bylevel':0.9,
    'subsample':0.9
}

In [ ]:
model = xgb.train(xgb_params, dtrain, num_boost_round=xgb_params['n_estimators'], evals=[(dval, 'val'), (dtrain, 'train')], early_stopping_rounds=int(xgb_params['n_estimators']/2))

In [ ]:
y_pred_proba = model.predict(dtest)

## Evaluation

Calculate important classification metrics and plot precision recall curve.

In [ ]:
ap = average_precision_score(y_test, y_pred_proba)
print("Average Precision: ", np.round(ap,2))

In [ ]:
fig = plot_ap(y_test, y_pred_proba)

In [ ]:
optimal_threshold, optimal_f1_score = get_optimal_f1_cutoff(y_test, y_pred_proba)
print("F1 Score: ", np.round(optimal_f1_score, 4))

In [ ]:
cm = get_confusion_matrix(y_test, y_pred_proba, optimal_threshold)
print("Confusion Matrix: \n", cm)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, y_pred_proba)